I. TASK: Extractive Question Answering

https://huggingface.co/course/chapter7/7?fw=pt



II. PRETRAINED MODEL: BERTa

https://huggingface.co/projecte-aina/roberta-base-ca-v2

https://github.com/projecte-aina/club

BERTa for Extractive-QA:

https://huggingface.co/projecte-aina/roberta-base-ca-cased-qa

III. DATA: CatalanQA


**Data Fields**

Follows (Rajpurkar, Pranav et al., 2016) for SQuAD v1 datasets:

- id (str): Unique ID assigned to the question.
- title (str): Title of the article.
- context (str): Article text.
- question (str): Question.
- answers (list): Answer to the question, containing:
 - text (str): Span text answering to the question.
 - answer_start Starting offset of the span text answering to the question.

**Data Splits**

- train.json: 17135 question/answer pairs
- dev.json: 2157 question/answer pairs
- test.json: 2135 question/answer pairs

https://huggingface.co/datasets/projecte-aina/catalanqa

#1. Setting the environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00


In [ ]:
import transformers

print(transformers.__version__)

4.27.4


In [ ]:
import tensorflow as tf
import torch
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla T4'

#2. Loading the data

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 20.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
# define the datasets ENGLISH

path_en = "/content/drive/My Drive/APP1/"

script_en = "/content/drive/My Drive/APP1/englishqa.py"

data_files_en = {"train": path_en + "train_en.json",
                 "dev": path_en + "train_en.json",
                 "test": path_en + "train_en.json"
             }

raw_datasets_en = load_dataset(script_en, data_files=data_files_en)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset englishqa downloaded and prepared to /root/.cache/huggingface/datasets/englishqa/default-8d52d12c8fa040fa/1.0.1/37a70f22e0fb8184e3e595c15adf307e6a04845d42e16fc65d7594cd5504bdb6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets_en

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86821
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86821
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86821
    })
})

In [ ]:
from transformers import set_seed

In [ ]:
transformers.set_seed(42)

In [ ]:
import random

# Get the indices of 17000 random examples from the English train split (to reduce the amount of computing power needed)
indices_en = random.sample(range(len(raw_datasets_en['train'])), 17000)

# Select the examples at the random indices
raw_datasets_en['train'] = raw_datasets_en['train'].select(indices_en)

In [ ]:
raw_datasets_en

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86821
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 86821
    })
})

In [ ]:
raw_datasets_en["train"][0]

{'id': '57334124d058e614000b57dd',
 'title': 'Portugal',
 'context': 'The governors of the taifas each proclaimed themselves Emir of their provinces and established diplomatic relations with the Christian kingdoms of the north. Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of the Abbadids poets. The Taifa period ended with the conquest of the Almoravids who came from Morocco in 1086 winning a decisive victory at the Battle of Sagrajas, followed a century later in 1147, after the second period of Taifa, by the Almohads, also from Marrakesh.',
 'question': 'Which which kingdoms did the taifas establish diplomatic relations?',
 'answers': [{'text': 'the Christian kingdoms of the north',
   'answer_start': 121}]}

In [ ]:
# define the datasets CATALAN

path_ca = "/content/drive/My Drive/APP1/"

script_ca = "/content/drive/My Drive/APP1/catalanqa.py"

data_files_ca = {"train": path_ca + "train_cat.json",
             "dev": path_ca + "dev_cat.json",
             "test": path_ca + "test_cat.json"}

raw_datasets_ca = load_dataset(script_ca, data_files=data_files_ca)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset catalanqa downloaded and prepared to /root/.cache/huggingface/datasets/catalanqa/default-242a678a5d510efa/1.0.1/d504905a7b9bd4ddb7a36a5cdda7527902cb929fa8bb25ca12d1adf40272fa8f. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets_ca

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17135
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2157
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2135
    })
})

In [ ]:
# Remove all examples from the Catalan train split for the zero-shot approach
indices = random.sample(range(len(raw_datasets_ca['train'])), 0)

# Select the examples at the random indices
raw_datasets_ca['train'] = raw_datasets_ca['train'].select(indices)

In [ ]:
raw_datasets_ca

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 0
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2157
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2135
    })
})

In [ ]:
# get a new dataset containing EN ['train'] dataset for training, and the CA ['validation] and ['test'] for validation and test

from datasets import DatasetDict, concatenate_datasets

train_en = raw_datasets_en["train"]
train_ca = raw_datasets_ca["train"]

train_enca = concatenate_datasets([train_en, train_ca])

# create a new DatasetDict with the three splits you want
raw_datasets = DatasetDict({
    "train": train_enca,
    "validation": raw_datasets_ca["validation"],
    "test": raw_datasets_ca["test"]
})

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17000
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2157
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 2135
    })
})

In [ ]:
# sample of English instance in train split
raw_datasets["train"][0]

{'id': '57334124d058e614000b57dd',
 'title': 'Portugal',
 'context': 'The governors of the taifas each proclaimed themselves Emir of their provinces and established diplomatic relations with the Christian kingdoms of the north. Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of the Abbadids poets. The Taifa period ended with the conquest of the Almoravids who came from Morocco in 1086 winning a decisive victory at the Battle of Sagrajas, followed a century later in 1147, after the second period of Taifa, by the Almohads, also from Marrakesh.',
 'question': 'Which which kingdoms did the taifas establish diplomatic relations?',
 'answers': [{'text': 'the Christian kingdoms of the north',
   'answer_start': 121}]}

In [ ]:
print("Sample instance:", raw_datasets["train"][0])

Sample instance: {'id': '57334124d058e614000b57dd', 'title': 'Portugal', 'context': 'The governors of the taifas each proclaimed themselves Emir of their provinces and established diplomatic relations with the Christian kingdoms of the north. Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of the Abbadids poets. The Taifa period ended with the conquest of the Almoravids who came from Morocco in 1086 winning a decisive victory at the Battle of Sagrajas, followed a century later in 1147, after the second period of Taifa, by the Almohads, also from Marrakesh.', 'question': 'Which which kingdoms did the taifas establish diplomatic relations?', 'answers': [{'text': 'the Christian kingdoms of the north', 'answer_start': 121}]}


In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  The governors of the taifas each proclaimed themselves Emir of their provinces and established diplomatic relations with the Christian kingdoms of the north. Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of the Abbadids poets. The Taifa period ended with the conquest of the Almoravids who came from Morocco in 1086 winning a decisive victory at the Battle of Sagrajas, followed a century later in 1147, after the second period of Taifa, by the Almohads, also from Marrakesh.
Question:  Which which kingdoms did the taifas establish diplomatic relations?
Answer:  [{'text': 'the Christian kingdoms of the north', 'answer_start': 121}]


#3. Preprocessing the training data

In [ ]:
from transformers import AutoTokenizer

In [ ]:
# choose the model_checkpoint for the tokeniser and pretrained model

model_checkpoint = "xlm-roberta-base"
#model_checkpoint = "projecte-aina/roberta-base-ca-v2"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
# make sure our model_checkpoint is supported for fast tokeniser
tokenizer.is_fast

True

In [ ]:
# check tokenisation with 1 sample
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'<s> Which which kingdoms did the taifas establish diplomatic relations?</s></s> The governors of the taifas each proclaimed themselves Emir of their provinces and established diplomatic relations with the Christian kingdoms of the north. Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of the Abbadids poets. The Taifa period ended with the conquest of the Almoravids who came from Morocco in 1086 winning a decisive victory at the Battle of Sagrajas, followed a century later in 1147, after the second period of Taifa, by the Almohads, also from Marrakesh.</s>'

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
)

for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

<s> Which which kingdoms did the taifas establish diplomatic relations?</s></s> The governors of the taifas each proclaimed themselves Emir of their provinces and established diplomatic relations with the Christian kingdoms of the north. Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville</s>
<s> Which which kingdoms did the taifas establish diplomatic relations?</s></s> Most of Portugal fell into the hands of the Taifa of Badajoz of the Aftasid Dynasty, and after a short spell of an ephemeral Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of the Abbadids poets. The Taifa period ended with the conquest of the Almoravids who came from Morocco in 1086 winning</s>
<s> Which which kingdoms did the taifas establish diplomatic relations?</s></s> Taifa of Lisbon in 1022, fell under the dominion of the Taifa of Seville of th

In [ ]:
inputs = tokenizer(
    question,
    context,
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [ ]:
inputs["overflow_to_sample_mapping"]

[0, 0, 0, 0]

In [ ]:
# check with several samples

inputs = tokenizer(
    raw_datasets["train"][2:6]["question"],
    raw_datasets["train"][2:6]["context"],
    max_length=100,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)

print(f"The 4 examples gave {len(inputs['input_ids'])} features.")
print(f"Here is where each comes from: {inputs['overflow_to_sample_mapping']}.")

The 4 examples gave 17 features.
Here is where each comes from: [0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3].


In [ ]:
answers = raw_datasets["train"][2:6]["answers"]
start_positions = []
end_positions = []

for i, offset in enumerate(inputs["offset_mapping"]):
    sample_idx = inputs["overflow_to_sample_mapping"][i]
    answer = answers[sample_idx]
    start_char = answer[0]["answer_start"]
    end_char = answer[0]["answer_start"] + len(answer[0]["text"])
    sequence_ids = inputs.sequence_ids(i)

    # Find the start and end of the context
    idx = 0
    while sequence_ids[idx] != 1:
        idx += 1
    context_start = idx
    while sequence_ids[idx] == 1:
        idx += 1
    context_end = idx - 1

    # If the answer is not fully inside the context, label is (0, 0)
    if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
        start_positions.append(0)
        end_positions.append(0)
    else:
        # Otherwise it's the start and end token positions
        idx = context_start
        while idx <= context_end and offset[idx][0] <= start_char:
            idx += 1
        start_positions.append(idx - 1)

        idx = context_end
        while idx >= context_start and offset[idx][1] >= end_char:
            idx -= 1
        end_positions.append(idx + 1)

start_positions, end_positions

([33, 20, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 74, 40],
 [34, 20, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 0, 0, 0, 79, 45])

In [ ]:
idx = 0
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx][0]["text"]

start = start_positions[idx]
end = end_positions[idx]
labeled_answer = tokenizer.decode(inputs["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")

Theoretical answer: Financial District, labels give: Financial District


In [ ]:
idx = 4
sample_idx = inputs["overflow_to_sample_mapping"][idx]
answer = answers[sample_idx][0]["text"]

decoded_example = tokenizer.decode(inputs["input_ids"][idx])
print(f"Theoretical answer: {answer}, decoded example: {decoded_example}")

Theoretical answer: Roman, decoded example: <s> Illyrian pirates caused two wars in the Balkans with what Empire?</s></s>, in an attempt to reduce Roman influence in the Balkans, Philip allied himself with Carthage after Hannibal had dealt the Romans a massive defeat at the Battle of Cannae (216 BC) during the Second Punic War. Forcing the Romans to fight on another front when they were at a nadir of manpower gained Philip the lasting enmity</s>


In [ ]:
# define function with all the steps seen above to preprocess the training set

max_length = 384
stride = 128


def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(train_dataset)

Map:   0%|          | 0/17000 [00:00<?, ? examples/s]

(17000, 17422)

#4. Preprocessing the validation data

In [ ]:
# preproceses the validation set

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(validation_dataset)

Map:   0%|          | 0/2157 [00:00<?, ? examples/s]

(2157, 2313)

#5. Fine-tuning the pretrained model

## Compute metrics

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.7 MB/s eta 0:00:00


In [ ]:
small_eval_set = raw_datasets["validation"].select(range(100))
trained_checkpoint = "distilbert-base-cased-distilled-squad"

tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
eval_set = small_eval_set.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

eval_set_for_model = eval_set.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained(trained_checkpoint).to(
    device
)

with torch.no_grad():
    outputs = trained_model(**batch)

In [ ]:
start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(eval_set):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in small_eval_set:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = eval_set["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
import evaluate

metric = evaluate.load("squad")

In [ ]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]} for ex in small_eval_set
]

In [ ]:
print(predicted_answers[0])
print(theoretical_answers[0])

{'id': '0.11139646731787067', 'prediction_text': 'però el 8 de setembre milers'}
{'id': '0.11139646731787067', 'answers': [{'text': 'milers', 'answer_start': 478}]}


In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 14.0, 'f1': 34.40517538017538}

In [ ]:
from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [ ]:
compute_metrics(start_logits, end_logits, eval_set, small_eval_set)

  0%|          | 0/100 [00:00<?, ?it/s]

{'exact_match': 14.0, 'f1': 34.40517538017538}

## Fine-tuning

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForQuestionAnswering: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream tas

In [ ]:
# we set the batch size for the Training Arguments
batch_size = 16

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    f"/content/drive/My Drive/APP1/xlm-roberta-qa-model-zero-shot",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=5e-5, # '2e-5', in the tutorial
    per_device_train_batch_size=batch_size, # '8', by default
    per_device_eval_batch_size=batch_size, # '8' by default
    num_train_epochs=5, # '3', in the tutorial
    weight_decay=0.01,
    fp16=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.322100
1000,1.456600
1500,1.077800


Step,Training Loss
500,2.322100
1000,1.456600
1500,1.077800
2000,0.995700
2500,0.779700
3000,0.646300
3500,0.569800
4000,0.436000
4500,0.375300
5000,0.282900


TrainOutput(global_step=5445, training_loss=0.8425048365737453, metrics={'train_runtime': 2809.3145, 'train_samples_per_second': 31.008, 'train_steps_per_second': 1.938, 'total_flos': 1.707117035945472e+16, 'train_loss': 0.8425048365737453, 'epoch': 5.0})

In [ ]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, raw_datasets["validation"])

  0%|          | 0/2157 [00:00<?, ?it/s]

{'exact_match': 53.12934631432545, 'f1': 78.65099502470918}

##OPTIONAL: customised training loop

In [ ]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

train_dataset.set_format("torch")
validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
validation_set.set_format("torch")

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    validation_set, collate_fn=default_data_collator, batch_size=8
)

In [ ]:
# we reinstantiate our model, to make sure we’re not continuing the fine-tuning from before 
# but starting from the initial pretrained model again

model_checkpoint = "projecte-aina/roberta-base-ca-v2"

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--projecte-aina--roberta-base-ca-v2/snapshots/772dfdf344529cf40009372bf14915c4a8aa1152/config.json
Model config RobertaConfig {
  "_name_or_path": "projecte-aina/roberta-base-ca-v2",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50262
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/mode

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 13.6 MB/s eta 0:00:00


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator() # 'Accelerator(fp16=True)' -> ERROR, unexpected argument
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
output_dir = "/content/drive/My Drive/Colab Notebooks/EHU/APP1/berta-qa-model_customised"

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

  0%|          | 0/6576 [00:00<?, ?it/s]

Evaluation!


  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/2157 [00:00<?, ?it/s]

Configuration saved in /content/drive/My Drive/Colab Notebooks/EHU/APP1/berta-qa-model_customised/config.json


...

  0%|          | 0/276 [00:00<?, ?it/s]

  0%|          | 0/2157 [00:00<?, ?it/s]

Configuration saved in /content/drive/My Drive/Colab Notebooks/EHU/APP1/berta-qa-model_customised/config.json


epoch 1: {'exact_match': 73.89893370421882, 'f1': 88.04678787944245}


Model weights saved in /content/drive/My Drive/Colab Notebooks/EHU/APP1/berta-qa-model_customised/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Colab Notebooks/EHU/APP1/berta-qa-model_customised/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Colab Notebooks/EHU/APP1/berta-qa-model_customised/special_tokens_map.json


#6. Using the fine-tuned model

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/drive/My Drive/APP1/xlm-roberta-qa-model-zero-shot/checkpoint-5445"
question_answerer = pipeline("question-answering", model=model_checkpoint)

In [ ]:
context = """
Noah Higón és de València, té 24 anys i des dels 12 que lluita per les malalties rares. 
Té dos llibres publicats narrant la seua història, forma part de diverses associacions i, 
ara serà el monument central de la Falla Tres Forques, el qual l’han titulat “Res és impossible”, 
frase que la jove té present al seu dia a dia.
"""
question = "Com es diu el monument de la Falla Tres Forques?"
answer = question_answerer(question=question, context=context)
print(answer)

{'score': 0.9723230004310608, 'start': 258, 'end': 278, 'answer': '“Res és impossible”,'}
